In [1]:
import numpy as np
import pickle
import nltk
from nltk import ne_chunk
from keras.preprocessing.sequence import pad_sequences

/home/gauravroy/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
raw_train = []
with open('/home/gauravroy/Desktop/slotfilling/ATIS_train.iob','r') as f:
    for line in f:
        raw_train.append(line)
        
raw_test = []
with open('/home/gauravroy/Desktop/slotfilling/ATIS_test.iob','r') as f:
    for line in f:
        raw_test.append(line)

# Preprocessing

In [3]:
train = []
replace_dic = {'\'d' : "would" , '\'s' : "is" , '\'t' : "not"}
for sentence in raw_train:
    sentence = sentence.split('\t')[0][4:-4]
    words = sentence.split(' ')
    new_sentence = ""
    for i in range(len(words)):
        word = words[i]
        word = word.split("'")[0]
        if(word == ''):
            print (word)
            continue;
        if(word in replace_dic):
            print(word)
            word = replace_dic[word];
        if(word.isdigit()):
            word = "number"
        new_sentence = new_sentence + " " + word
    train.append(new_sentence[1:])

In [4]:
train[10]

'show me the flights from dallas to san francisco'

In [5]:
test = []
replace_dic = {'\'d' : "would" , '\'s' : "is" , '\'t' : "not"}
for sentence in raw_test:
    sentence = sentence.split('\t')[0][4:-4]
    words = sentence.split(' ')
    new_sentence = ""
    for i in range(len(words)):
        word = words[i]
        word = word.split("'")[0]
        if(word == ''):
            continue;
        if(word in replace_dic):
            word = replace_dic[word];
        if(word.isdigit()):
            word = "number"
        new_sentence = new_sentence + " " + word
    test.append(new_sentence[1:])

In [6]:
test[10]

'find a flight from memphis to tacoma dinner'

In [7]:
 with open('/home/gauravroy/Desktop/slotfilling/train_data.pickle', 'wb') as fp:
     pickle.dump(train,fp)
 fp.close()

 with open('/home/gauravroy/Desktop/slotfilling/test_data.pickle', 'wb') as fp:
     pickle.dump(test,fp)
 fp.close()

In [8]:
with open('/home/gauravroy/Desktop/slotfilling/train_data.pickle', 'rb') as fp:
    train = pickle.load(fp)
fp.close()

with open('/home/gauravroy/Desktop/slotfilling/test_data.pickle', 'rb') as fp:
    test = pickle.load(fp)
fp.close()

In [9]:
test

['i would like to find a flight from charlotte to las vegas that makes a stop in st. louis',
 'on april first i need a ticket from tacoma to san jose departing before number am',
 'on april first i need a flight going from phoenix to san diego',
 'i would like a flight traveling one way from phoenix to san diego on april first',
 'i would like a flight from orlando to salt lake city for april first on delta airlines',
 'i need a flight from toronto to newark one way leaving wednesday evening or thursday morning',
 'monday morning i would like to fly from columbus to indianapolis',
 'on wednesday april sixth i would like to fly from long beach to columbus after number pm',
 'after number pm on wednesday april sixth i would like to fly from long beach to columbus',
 'are there any flights from long beach to columbus on wednesday april sixth',
 'find a flight from memphis to tacoma dinner',
 'on next wednesday flight from kansas city to chicago should arrive in chicago around number pm re

In [10]:
raw_test[0]

'BOS i would like to find a flight from charlotte to las vegas that makes a stop in st. louis EOS\tO O O O O O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name O O O O O B-stoploc.city_name I-stoploc.city_name atis_flight\n'

In [11]:
raw = raw_train + raw_test

# Dictionary Generation

In [12]:
def check(word):
    replace_dic = {'\'d' : "would" , '\'s' : "is" , '\'t' : "not"}
    word = word.split("'")[0]
    if(word == ''):
        return word
    if(word in replace_dic):
        word = replace_dic[word];
    if(word.isdigit()):
        word = "number"
    return word

In [13]:
slot_dict = dict()
word_dict = dict()

i_s = 0
i_w = 1

for utterance in raw:
    t = utterance.split('\t')
    words = t[0].strip().split(' ')
    slots = t[1].strip().split(' ')
    n = len(words)
    for i in range(n):
        word = check(words[i])
        if(word not in word_dict):
            word_dict[word] = i_w
            i_w = i_w + 1
    for i in range(n-1):
        if(slots[i] not in slot_dict):
            slot_dict[slots[i]] = i_s
            i_s = i_s + 1

In [14]:
slot_dict

{'O': 0,
 'B-fromloc.city_name': 1,
 'B-depart_time.time': 2,
 'I-depart_time.time': 3,
 'B-toloc.city_name': 4,
 'B-arrive_time.time': 5,
 'B-arrive_time.period_of_day': 6,
 'B-depart_date.day_name': 7,
 'B-depart_time.period_of_day': 8,
 'B-flight_time': 9,
 'I-flight_time': 10,
 'I-fromloc.city_name': 11,
 'B-cost_relative': 12,
 'B-round_trip': 13,
 'I-round_trip': 14,
 'B-fare_amount': 15,
 'I-fare_amount': 16,
 'B-depart_date.today_relative': 17,
 'I-toloc.city_name': 18,
 'B-city_name': 19,
 'B-stoploc.city_name': 20,
 'B-toloc.airport_code': 21,
 'B-depart_time.time_relative': 22,
 'B-class_type': 23,
 'I-class_type': 24,
 'B-depart_date.date_relative': 25,
 'B-airline_name': 26,
 'I-airline_name': 27,
 'B-arrive_time.time_relative': 28,
 'B-depart_time.start_time': 29,
 'I-depart_time.start_time': 30,
 'B-depart_time.end_time': 31,
 'I-depart_time.end_time': 32,
 'B-fromloc.airport_name': 33,
 'I-fromloc.airport_name': 34,
 'B-toloc.state_name': 35,
 'B-depart_date.day_number'

In [15]:
len(slot_dict)

127

In [16]:
word_dict

{'BOS': 1,
 'i': 2,
 'want': 3,
 'to': 4,
 'fly': 5,
 'from': 6,
 'boston': 7,
 'at': 8,
 'number': 9,
 'am': 10,
 'and': 11,
 'arrive': 12,
 'in': 13,
 'denver': 14,
 'the': 15,
 'morning': 16,
 'EOS': 17,
 'what': 18,
 'flights': 19,
 'are': 20,
 'available': 21,
 'pittsburgh': 22,
 'baltimore': 23,
 'on': 24,
 'thursday': 25,
 'is': 26,
 'arrival': 27,
 'time': 28,
 'san': 29,
 'francisco': 30,
 'for': 31,
 'flight': 32,
 'leaving': 33,
 'washington': 34,
 'cheapest': 35,
 'airfare': 36,
 'tacoma': 37,
 'orlando': 38,
 'round': 39,
 'trip': 40,
 'fares': 41,
 'philadelphia': 42,
 'under': 43,
 'dollars': 44,
 'need': 45,
 'a': 46,
 'tomorrow': 47,
 'columbus': 48,
 'minneapolis': 49,
 'kind': 50,
 'of': 51,
 'aircraft': 52,
 'used': 53,
 'cleveland': 54,
 'dallas': 55,
 'show': 56,
 'me': 57,
 'los': 58,
 'angeles': 59,
 'all': 60,
 'ground': 61,
 'transportation': 62,
 'diego': 63,
 'newark': 64,
 'by': 65,
 'way': 66,
 'houston': 67,
 'airport': 68,
 'bwi': 69,
 'after': 70,
 'pm'

In [17]:
dicts = [slot_dict,word_dict]

In [18]:
dicts

[{'O': 0,
  'B-fromloc.city_name': 1,
  'B-depart_time.time': 2,
  'I-depart_time.time': 3,
  'B-toloc.city_name': 4,
  'B-arrive_time.time': 5,
  'B-arrive_time.period_of_day': 6,
  'B-depart_date.day_name': 7,
  'B-depart_time.period_of_day': 8,
  'B-flight_time': 9,
  'I-flight_time': 10,
  'I-fromloc.city_name': 11,
  'B-cost_relative': 12,
  'B-round_trip': 13,
  'I-round_trip': 14,
  'B-fare_amount': 15,
  'I-fare_amount': 16,
  'B-depart_date.today_relative': 17,
  'I-toloc.city_name': 18,
  'B-city_name': 19,
  'B-stoploc.city_name': 20,
  'B-toloc.airport_code': 21,
  'B-depart_time.time_relative': 22,
  'B-class_type': 23,
  'I-class_type': 24,
  'B-depart_date.date_relative': 25,
  'B-airline_name': 26,
  'I-airline_name': 27,
  'B-arrive_time.time_relative': 28,
  'B-depart_time.start_time': 29,
  'I-depart_time.start_time': 30,
  'B-depart_time.end_time': 31,
  'I-depart_time.end_time': 32,
  'B-fromloc.airport_name': 33,
  'I-fromloc.airport_name': 34,
  'B-toloc.state_na

In [19]:
len(dicts)

2

In [20]:
with open('/home/gauravroy/Desktop/slotfilling//slot_word_dict.pickle', 'wb') as fp:
     pickle.dump(dicts,fp)
fp.close()

In [21]:
with open('/home/gauravroy/Desktop/slotfilling//slot_word_dict.pickle', 'rb') as fp:
    dicts = pickle.load(fp)
fp.close()

In [22]:
slot_dict,word_dict = dicts

In [23]:
len(word_dict)

766

# Sequence Making

In [24]:
def make_sequences(data,word_dict):
    sequences = list()
    for sentence in data:
        sequence = list()
        words = sentence.split(' ')
        for word in words:
                sequence.append(word_dict[word])
        sequences.append(sequence)
    return sequences

In [25]:
train[0]

'i want to fly from boston at number am and arrive in denver at number in the morning'

In [26]:
train_sequence = make_sequences(train,word_dict)
test_sequence = make_sequences(test,word_dict)

In [27]:
train_sequence

[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 8, 9, 13, 15, 16],
 [18, 19, 20, 21, 6, 22, 4, 23, 24, 25, 16],
 [18, 26, 15, 27, 28, 13, 29, 30, 31, 15, 9, 10, 32, 33, 34],
 [35, 36, 6, 37, 4, 38],
 [39, 40, 41, 6, 22, 4, 42, 43, 9, 44],
 [2, 45, 46, 32, 47, 6, 48, 4, 49],
 [18, 50, 51, 52, 26, 53, 24, 46, 32, 6, 54, 4, 55],
 [56, 57, 15, 19, 6, 22, 4, 58, 59, 24, 25],
 [60, 19, 6, 7, 4, 34],
 [18, 50, 51, 61, 62, 26, 21, 13, 14],
 [56, 57, 15, 19, 6, 55, 4, 29, 30],
 [56, 57, 15, 19, 6, 29, 63, 4, 64, 65, 66, 51, 67],
 [18, 15, 68, 8, 38],
 [18, 26, 15, 35, 32, 6, 7, 4, 69],
 [60, 19, 4, 23, 70, 9, 71],
 [56, 57, 15, 72, 73, 41, 6, 7, 4, 14],
 [56, 57, 15, 61, 62, 13, 14],
 [60, 19, 6, 14, 4, 22, 33, 70, 9, 71, 11, 74, 9, 71],
 [2, 45, 75, 24, 19, 31, 76, 33, 23, 31, 55, 55, 4, 7, 11, 7, 4, 23],
 [77, 78, 57, 15, 19, 6, 7, 4, 22, 24, 25, 51, 79, 80],
 [2, 81, 82, 4, 5, 6, 14, 4, 22, 24, 83, 84],
 [56, 57, 15, 19, 6, 29, 63, 4, 64],
 [77, 85, 60, 72, 73, 19, 24, 83, 6, 14, 4, 23],
 [18,

In [28]:
len(raw_train)

4978

In [29]:
with open('/home/gauravroy/Desktop/slotfilling/train_sequence.pickle', 'wb') as fp:
     pickle.dump(train_sequence,fp)
fp.close()

with open('/home/gauravroy/Desktop/slotfilling/test_sequence.pickle', 'wb') as fp:
     pickle.dump(test_sequence,fp)
fp.close()

In [30]:
with open('/home/gauravroy/Desktop/slotfilling/train_sequence.pickle', 'rb') as fp:
    train_sequence = pickle.load(fp)
fp.close()

with open('/home/gauravroy/Desktop/slotfilling/test_sequence.pickle', 'rb') as fp:
    test_sequence = pickle.load(fp)
fp.close()

In [31]:
sequences = train_sequence + test_sequence

In [32]:
m = 0
lens = list()
for sequence in train_sequence + test_sequence:
    lens.append(len(sequence))
    m = max(len(sequence),m)
    
print (m)

46


In [33]:
train_sequence = pad_sequences(train_sequence, maxlen = 46, padding='post')
test_sequence = pad_sequences(test_sequence, maxlen = 46, padding='post') 

In [34]:
train_sequence

array([[  2,   3,   4, ...,   0,   0,   0],
       [ 18,  19,  20, ...,   0,   0,   0],
       [ 18,  26,  15, ...,   0,   0,   0],
       ...,
       [ 94,  84,   5, ...,   0,   0,   0],
       [152, 297,   5, ...,   0,   0,   0],
       [ 26, 217,  46, ...,   0,   0,   0]], dtype=int32)

In [35]:
from matplotlib import pyplot
pyplot.hist(lens,46)
pyplot.show()

## Output in One-Hot form(Slot)

In [36]:
utterance = raw_train[0]
slots = utterance.split('\t')[1].strip().split(' ')[:-1]
slots = t
slots

['BOS find me a flight that flies from memphis to tacoma EOS',
 'O O O O O O O O B-fromloc.city_name O B-toloc.city_name atis_flight\n']

In [37]:
n = len(slot_dict)

y_train_slot = list()
for utterance in raw_train:
    outs = list()
    slots = utterance.split('\t')[1].strip().split(' ')[:-1]
    for slot in slots:
        out = np.zeros(n)
        out = list(out)
        out[slot_dict[slot]] = 1
        outs.append(out) 
    y_train_slot.append(outs)
    
y_test_slot = list()
for utterance in raw_test:
    outs = list()
    slots = utterance.split('\t')[1].strip().split(' ')[:-1]
    for slot in slots:
        out = np.zeros(n)
        out = list(out)
        out[slot_dict[slot]] = 1
        outs.append(out) 
    y_test_slot.append(outs)


In [38]:
with open('/home/gauravroy/Desktop/slotfilling/y_train_slot.pickle', 'wb') as fp:
    pickle.dump(y_train_slot,fp, protocol = 2)
fp.close()

with open('/home/gauravroy/Desktop/slotfilling//y_test_slot.pickle', 'wb') as fp:
    pickle.dump(y_test_slot,fp, protocol = 2)
fp.close()

# POS Train_set

In [39]:
pos_dict = dict()
i = 0
train_tag=[]
for utterance in train:
    tags = nltk.pos_tag(utterance.split(' '))
    #x.append(tags)
    y=[]
    for tag in tags:
        tag = tag[1]
        y.append(tag)
    train_tag.append(y)

In [40]:
train_tag

[['NN',
  'VBP',
  'TO',
  'VB',
  'IN',
  'NN',
  'IN',
  'NN',
  'VBP',
  'CC',
  'VBP',
  'IN',
  'NN',
  'IN',
  'NN',
  'IN',
  'DT',
  'NN'],
 ['WP', 'NNS', 'VBP', 'JJ', 'IN', 'NN', 'TO', 'VB', 'IN', 'JJ', 'NN'],
 ['WP',
  'VBZ',
  'DT',
  'JJ',
  'NN',
  'IN',
  'JJ',
  'NN',
  'IN',
  'DT',
  'NN',
  'VBP',
  'NN',
  'VBG',
  'NN'],
 ['JJS', 'NN', 'IN', 'NN', 'TO', 'VB'],
 ['NN', 'NN', 'NNS', 'IN', 'NN', 'TO', 'VB', 'IN', 'NN', 'NNS'],
 ['NNS', 'VBP', 'DT', 'NN', 'NN', 'IN', 'NN', 'TO', 'VB'],
 ['WP',
  'NN',
  'IN',
  'NN',
  'VBZ',
  'VBN',
  'IN',
  'DT',
  'NN',
  'IN',
  'NN',
  'TO',
  'VB'],
 ['VB', 'PRP', 'DT', 'NNS', 'IN', 'NN', 'TO', 'VB', 'NNS', 'IN', 'NN'],
 ['DT', 'NNS', 'IN', 'NN', 'TO', 'VB'],
 ['WP', 'NN', 'IN', 'NN', 'NN', 'VBZ', 'JJ', 'IN', 'NN'],
 ['VB', 'PRP', 'DT', 'NNS', 'IN', 'NN', 'TO', 'VB', 'NN'],
 ['VB',
  'PRP',
  'DT',
  'NNS',
  'IN',
  'JJ',
  'NN',
  'TO',
  'VB',
  'IN',
  'NN',
  'IN',
  'NN'],
 ['WP', 'DT', 'NN', 'IN', 'NN'],
 ['WP', 'VBZ', 'D

# POS for Test_set

In [41]:
pos_dict = dict()
i = 0
test_tag=[]
for utterance in test:
    tags = nltk.pos_tag(utterance.split(' '))
    #x.append(tags)
    y=[]
    for tag in tags:
        tag = tag[1]
        y.append(tag)
    test_tag.append(y)

In [42]:
test_tag

[['NN',
  'MD',
  'VB',
  'TO',
  'VB',
  'DT',
  'NN',
  'IN',
  'NN',
  'TO',
  'VB',
  'NN',
  'WDT',
  'VBZ',
  'DT',
  'NN',
  'IN',
  'NN',
  'NN'],
 ['IN',
  'NN',
  'RB',
  'RB',
  'VBP',
  'DT',
  'NN',
  'IN',
  'NN',
  'TO',
  'VB',
  'JJ',
  'VBG',
  'IN',
  'NN',
  'VBP'],
 ['IN',
  'NN',
  'RB',
  'RB',
  'VBP',
  'DT',
  'NN',
  'VBG',
  'IN',
  'NN',
  'TO',
  'VB',
  'NN'],
 ['NN',
  'MD',
  'VB',
  'DT',
  'NN',
  'VBG',
  'CD',
  'NN',
  'IN',
  'NN',
  'TO',
  'VB',
  'NN',
  'IN',
  'NN',
  'RB'],
 ['NN',
  'MD',
  'VB',
  'DT',
  'NN',
  'IN',
  'NN',
  'TO',
  'VB',
  'JJ',
  'NN',
  'IN',
  'NN',
  'RB',
  'IN',
  'NN',
  'NNS'],
 ['NNS',
  'VBP',
  'DT',
  'NN',
  'IN',
  'NN',
  'TO',
  'VB',
  'CD',
  'NN',
  'VBG',
  'JJ',
  'NN',
  'CC',
  'JJ',
  'NN'],
 ['JJ', 'NN', 'NN', 'MD', 'VB', 'TO', 'VB', 'IN', 'NN', 'TO', 'VB'],
 ['IN',
  'JJ',
  'IN',
  'JJ',
  'NN',
  'MD',
  'VB',
  'TO',
  'VB',
  'IN',
  'RB',
  'NN',
  'TO',
  'VB',
  'IN',
  'NN',
  'NN'],


### Make POS_dict

In [43]:
pos_dict = dict()
i = 0

for utterance in train + test:
    tags = nltk.pos_tag(utterance.split(' '))
    for tag in tags:
        tag = tag[1]
        if(tag not in pos_dict):
            pos_dict[tag] = i
            i += 1

In [44]:
pos_dict

{'NN': 0,
 'VBP': 1,
 'TO': 2,
 'VB': 3,
 'IN': 4,
 'CC': 5,
 'DT': 6,
 'WP': 7,
 'NNS': 8,
 'JJ': 9,
 'VBZ': 10,
 'VBG': 11,
 'JJS': 12,
 'VBN': 13,
 'PRP': 14,
 'MD': 15,
 'WDT': 16,
 'RB': 17,
 'WRB': 18,
 'RBS': 19,
 'PDT': 20,
 'CD': 21,
 'EX': 22,
 'VBD': 23,
 'RP': 24,
 'NNP': 25,
 'JJR': 26,
 'PRP$': 27,
 'FW': 28,
 'RBR': 29,
 'UH': 30}

# Sequence Making Of POS_Tag

In [45]:
def make_sequences(data,pos_dict):
    sequences = list()
    for tags in data:
        sequence = list()
        #words = sentence.split(' ')
        for i in range(len(tags)):
                sequence.append(pos_dict[tags[i]])
        sequences.append(sequence)
    return sequences

In [46]:
train_sequence1 = make_sequences(train_tag,pos_dict)
test_sequence1 = make_sequences(test_tag,pos_dict)

In [47]:
pos_dict['NN']

0

In [48]:
m = 0
lens = list()
for sequence in train_sequence1 + test_sequence1:
    lens.append(len(sequence))
    m = max(len(sequence),m)
    
print (m)

46


In [49]:
train_sequence1 = pad_sequences(train_sequence1, maxlen = 46, padding='post')
test_sequence1 = pad_sequences(test_sequence1, maxlen = 46, padding='post') 

In [50]:
train_sequence1

array([[ 0,  1,  2, ...,  0,  0,  0],
       [ 7,  8,  1, ...,  0,  0,  0],
       [ 7, 10,  6, ...,  0,  0,  0],
       ...,
       [16, 10,  0, ...,  0,  0,  0],
       [10,  9,  3, ...,  0,  0,  0],
       [10, 22,  6, ...,  0,  0,  0]], dtype=int32)

In [51]:
with open('/home/gauravroy/Desktop/slotfilling/train_sequence1.pickle', 'wb') as fp:
    pickle.dump(train_sequence1,fp)
fp.close()

with open('/home/gauravroy/Desktop/slotfilling/test_sequence1.pickle', 'wb') as fp:
     pickle.dump(test_sequence1,fp)
fp.close()

# Embedding Of POS_Tag

In [52]:
d = list(pos_dict.keys())

In [53]:
d

['NN',
 'VBP',
 'TO',
 'VB',
 'IN',
 'CC',
 'DT',
 'WP',
 'NNS',
 'JJ',
 'VBZ',
 'VBG',
 'JJS',
 'VBN',
 'PRP',
 'MD',
 'WDT',
 'RB',
 'WRB',
 'RBS',
 'PDT',
 'CD',
 'EX',
 'VBD',
 'RP',
 'NNP',
 'JJR',
 'PRP$',
 'FW',
 'RBR',
 'UH']

In [54]:
a = []
a.append(d)

In [55]:
from gensim.models import Word2Vec
model = Word2Vec(a, size = 5, min_count=1)

In [56]:
print(model)

Word2Vec(vocab=31, size=5, alpha=0.025)


In [57]:
words = list(model.wv.vocab)

In [58]:
print(model['NNP'])

[ 0.06929069  0.01389781  0.022944   -0.04263892 -0.05461409]


/home/gauravroy/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [59]:
nb_words = len(pos_dict)
EMBEDDING_DIM = 5
embedding_matrix_pos = np.zeros((nb_words+1, EMBEDDING_DIM))

i = 0
x_g = x_w = 0
for tag in pos_dict:
    try:
        embedding_matrix_pos[pos_dict[tag]][:31] = model[tag]
    except KeyError:
        x_g += 1

/home/gauravroy/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [60]:
np.shape(embedding_matrix_pos)

(32, 5)

In [61]:
with open('/home/gauravroy/Desktop/slotfilling/embedding_matrix_pos.pickle', 'wb') as fp:
     pickle.dump(embedding_matrix_pos, fp)
fp.close()

In [62]:
len(pos_dict)

31

In [63]:
n = len(pos_dict)
max_len = 46

x_train_pos = list()
for utterance in train:
    outs = np.zeros((max_len,len(pos_dict)))
#     outs = list(outs)
    tags = nltk.pos_tag(utterance.split(' '))
    for i in range(len(tags)):
        outs[i][pos_dict[tags[i][1]]] = 1
    x_train_pos.append(outs)
    
x_test_pos = list()
for utterance in test:
    outs = np.zeros((max_len,len(pos_dict)))
#     outs = list(outs)
    tags = nltk.pos_tag(utterance.split(' '))
    for i in range(len(tags)):
        outs[i][pos_dict[tags[i][1]]] = 1
    x_test_pos.append(outs)

In [64]:
x_train_pos

[array([[1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]), array([

In [65]:
with open('/home/gauravroy/Desktop/slotfilling/x_train_pos.pickle', 'wb') as fp:
    pickle.dump(x_train_pos,fp)
fp.close()

with open('/home/gauravroy/Desktop/slotfilling//x_test_pos.pickle', 'wb') as fp:
    pickle.dump(x_test_pos,fp)
fp.close()

In [66]:
len(pos_dict)

31

In [67]:
train[3]

'cheapest airfare from tacoma to orlando'

In [68]:
x_train_pos[2][3]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])